In [ ]:
import pandas as pd
import json
import os

# Path to the data directory
VERSION = 'v1.0'
data_dir = f'data/{VERSION}'

# Get all jsonl files from the directory
try:
    files_to_process = [os.path.join(data_dir, f) for f in os.listdir(data_dir) if f.endswith('.jsonl')]
except FileNotFoundError:
    print(f"Error: The directory '{data_dir}' was not found.")
    files_to_process = []

# List to hold all the data
all_data = []

# Loop through each file
if files_to_process:
    for file_path in files_to_process:
        file_name = os.path.basename(file_path)
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                for line in f:
                    try:
                        # Load the json object from the line
                        data = json.loads(line)
                        # Add the file name to the data
                        data['file_name'] = file_name
                        # Append to our list
                        all_data.append(data)
                    except json.JSONDecodeError:
                        print(f"Warning: Could not decode JSON from a line in {file_name}")
        except FileNotFoundError:
            print(f"Warning: File not found {file_path}")
        except Exception as e:
            print(f"An error occurred while processing {file_path}: {e}")

# Create a pandas DataFrame
if all_data:
    df = pd.DataFrame(all_data)

    # Normalize the 'persona' column if it exists
    if 'persona' in df.columns:
        # Handle cases where persona might be missing in some rows
        persona_data = df['persona'].apply(lambda x: x if isinstance(x, dict) else {}).tolist()
        persona_df = pd.json_normalize(persona_data)
        persona_df = persona_df.add_prefix('persona.')
        
        # Drop the original 'persona' column and join the new one
        df = df.drop(columns=['persona']).join(persona_df)

    # Define column configuration. This is the single source of truth for column names, order, and formatting.
    # The 'source' key refers to the original column name in the data.
    # The 'name' key is the desired display name in the Excel file.
    header_config = [
        {'source': 'file_name', 'name': 'نام فایل', 'width': 15.07, 'horizontal': 'left', 'vertical': 'center'},
        {'source': 'model', 'name': 'مدل', 'width': 8.59, 'horizontal': 'center', 'vertical': 'center'},
        {'source': 'id', 'name': 'ID', 'width': 8.59, 'horizontal': 'general', 'vertical': 'center'},
        {'source': 'persona.id', 'name': 'پرسونا ID', 'width': 8.59, 'horizontal': 'center', 'vertical': 'center'},
        {'source': 'persona.name', 'name': 'نام پرسونا', 'width': 19.11, 'horizontal': 'right', 'vertical': 'center'},
        {'source': 'persona.age', 'name': 'سن', 'width': 8.59, 'horizontal': 'center', 'vertical': 'center'},
        {'source': 'persona.gender', 'name': 'جنسیت', 'width': 13.0, 'horizontal': 'center', 'vertical': 'center'},
        {'source': 'persona.level_of_education', 'name': 'تحصیلات', 'width': 12.5, 'horizontal': 'center', 'vertical': 'center'},
        {'source': 'persona.occupation', 'name': 'شغل سابق', 'width': 20.95, 'horizontal': 'center', 'vertical': 'center'},
        {'source': 'persona.financial_status', 'name': 'وضعیت مالی', 'width': 16.42, 'horizontal': 'center', 'vertical': 'center'},
        {'source': 'persona.marital_status', 'name': 'وضعیت تاهل', 'width': 21.81, 'horizontal': 'center', 'vertical': 'center'},
        {'source': 'persona.personality_traits', 'name': 'صفات شخصیتی', 'width': 46.69, 'horizontal': 'general', 'vertical': 'bottom'},
        {'source': 'persona.background', 'name': 'پیشینه و سبک زندگی', 'width': 55.27, 'horizontal': 'right', 'vertical': 'top'},
        {'source': 'persona.religion', 'name': 'مذهب', 'width': 17.64, 'horizontal': 'center', 'vertical': 'center'},
        {'source': 'persona.spiritual_health_loss_of_independence', 'name': 'سلامت معنوی در موقعیت کاهش استقلال', 'width': 68.63, 'horizontal': 'general', 'vertical': 'bottom'},
        {'source': 'persona.spiritual_health_loss_of_social_activity', 'name': 'سلامت معنوی در موقعیت کاهش کنشگری اجتماعی', 'width': 77.21, 'horizontal': 'general', 'vertical': 'bottom'},
        {'source': 'persona.spiritual_health_physical_health_and_sexual_issues', 'name': 'سلامت معنوی با وجود کاهش سلامت جسمی و مشکلات جنسی', 'width': 60.66, 'horizontal': 'general', 'vertical': 'bottom'},
        {'source': 'persona.spiritual_health_loss_of_close_ones_and_fear_of_death', 'name': 'سلامت معنوی هنگام مرگ نزدیکان و ترس از مرگ', 'width': 65.93, 'horizontal': 'general', 'vertical': 'bottom'},
        {'source': 'persona.spiritual_health_loss_of_family_connections', 'name': 'سلامت معنوی در موقعیت کاهش ارتباطات خانوادگی', 'width': 49.51, 'horizontal': 'general', 'vertical': 'bottom'},
        {'source': 'persona.spiritual_health_lifestyle_changes', 'name': 'سلامت معنوی در شرایط تغییر سبک زندگی', 'width': 51.96, 'horizontal': 'general', 'vertical': 'bottom'},
        {'source': 'persona.spiritual_health_loss_of_income', 'name': 'سلامت معنوی در موقعیت کاهش درآمد مالی', 'width': 35.17, 'horizontal': 'general', 'vertical': 'bottom'},
        {'source': 'persona.spiritual_health_loss_of_aspiration', 'name': 'سلامت معنوی در موقعیت بیآرمانی', 'width': 40.56, 'horizontal': 'general', 'vertical': 'bottom'},
        {'source': 'persona.spiritual_health_life_integrity', 'name': 'سلامت معنوی در مواجهه با نیاز به یکپارچگی زندگی', 'width': 51.22, 'horizontal': 'general', 'vertical': 'bottom'},
        {'source': 'question', 'name': 'سوال', 'width': 59.44, 'horizontal': 'general', 'vertical': 'top'},
        {'source': 'answer', 'name': 'پاسخ', 'width': 91.42, 'horizontal': 'right', 'vertical': 'bottom'},
    ]

    # Create header map from config and rename columns
    header_map = {item['source']: item['name'] for item in header_config if 'source' in item}
    df.rename(columns=header_map, inplace=True)

    # Convert personality_traits arrays to comma-separated strings
    persona_traits_col = 'صفات شخصیتی'  # Persian name for personality traits
    if persona_traits_col in df.columns:
        df[persona_traits_col] = df[persona_traits_col].apply(
            lambda x: ', '.join(x) if isinstance(x, list) else str(x) if x is not None else ''
        )

    # Reorder columns according to the header_config list
    ordered_col_names = [c['name'] for c in header_config]
    existing_cols = [col for col in ordered_col_names if col in df.columns]
    other_cols = [col for col in df.columns if col not in existing_cols]
    df = df[existing_cols + other_cols]

    # Write to Excel with custom column widths and alignment
    output_path = f'../combined_data_{VERSION}.xlsx'
    try:
        with pd.ExcelWriter(output_path, engine='xlsxwriter') as writer:
            df.to_excel(writer, index=False, sheet_name='Sheet1')

            workbook = writer.book
            worksheet = writer.sheets['Sheet1']

            # Define formats
            header_format = workbook.add_format({
                'bold': True,
                'bg_color': '#D3D3D3',  # Light gray background
                'border': 1,
                'align': 'center',
                'valign': 'vcenter',
                'text_wrap': True,
                'font_name': 'Vazirmatn',
            })

            # Create a mapping from column name to config for easy lookup
            config_map = {c['name']: c for c in header_config}

            # Apply header formatting
            for i, col_name in enumerate(df.columns):
                worksheet.write(0, i, col_name, header_format)

            # Apply formats to columns and data cells
            for i, col_name in enumerate(df.columns):
                config = config_map.get(col_name, {})
                
                cell_format = workbook.add_format({
                    'align': config.get('horizontal', 'left'),
                    'valign': config.get('vertical', 'vcenter'),
                    'text_wrap': True,
                    'border': 1,  # Black border around all cells
                    'font_name': 'Vazirmatn',
                })
                
                worksheet.set_column(i, i, config.get('width', 15), cell_format)

        print(f"Successfully created {output_path}")
    except Exception as e:
        print(f"Error writing to Excel file: {e}")
else:
    print("No data was processed to create an Excel file.")



Successfully created ../combined_data_v1.0.xlsx
